## part1

year                
month    
day_of_month
day_of_week
fl_date

In [34]:
import numpy as np
import pandas as pd

In [ ]:
#path = r"D:\py\5243\project1\flight_data_2024.csv"
#df = pd.read_csv(path)

df = pd.read_csv("flight_data_2024.csv")

cols1 = ["year", "month", "day_of_month", "day_of_week", "fl_date"]

C:\Users\ZYJ\AppData\Local\Temp\ipykernel_11840\3256857495.py:2: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [ ]:
df1 = df.copy()

In [ ]:
# get basic info for dataset 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7079081 entries, 0 to 7079080
Data columns (total 35 columns):
 #   Column               Dtype  
---  ------               -----  
 0   year                 int64  
 1   month                int64  
 2   day_of_month         int64  
 3   day_of_week          int64  
 4   fl_date              object 
 5   op_unique_carrier    object 
 6   op_carrier_fl_num    float64
 7   origin               object 
 8   origin_city_name     object 
 9   origin_state_nm      object 
 10  dest                 object 
 11  dest_city_name       object 
 12  dest_state_nm        object 
 13  crs_dep_time         int64  
 14  dep_time             float64
 15  dep_delay            float64
 16  taxi_out             float64
 17  wheels_off           float64
 18  wheels_on            float64
 19  taxi_in              float64
 20  crs_arr_time         int64  
 21  arr_time             float64
 22  arr_delay            float64
 23  cancelled            int64  
 24

In [ ]:
# check missing values for each column

print("Missing values per column:")
print(df.isna().sum())

print("\nNumber of duplicated rows:")
print(df.duplicated().sum())


Missing values per column:
year                         0
month                        0
day_of_month                 0
day_of_week                  0
fl_date                      0
op_unique_carrier            0
op_carrier_fl_num            1
origin                       0
origin_city_name             0
origin_state_nm              0
dest                         0
dest_city_name               0
dest_state_nm                0
crs_dep_time                 0
dep_time                 92659
dep_delay                92970
taxi_out                 95734
wheels_off               95734
wheels_on                97856
taxi_in                  97856
crs_arr_time                 0
arr_time                 97854
arr_delay               113814
cancelled                    0
cancellation_code      6982766
diverted                     0
crs_elapsed_time             1
actual_elapsed_time     113814
air_time                113814
distance                     0
carrier_delay                0
weather_dela

In [38]:
print("Missing values:")
print(df[cols1].isna().sum())

Missing values:
year            0
month           0
day_of_month    0
day_of_week     0
fl_date         0
dtype: int64


In [39]:
df.shape

(7079081, 35)

In [40]:
abnormal_month = df[(df["month"] < 1) | (df["month"] > 12)]
abnormal_day = df[(df["day_of_month"] < 1) | (df["day_of_month"] > 31)]
abnormal_week = df[(df["day_of_week"] < 1) | (df["day_of_week"] > 7)]

print("\nAbnormal month:", len(abnormal_month))
print("Abnormal day_of_month:", len(abnormal_day))
print("Abnormal day_of_week:", len(abnormal_week))


Abnormal month: 0
Abnormal day_of_month: 0
Abnormal day_of_week: 0


In [41]:
df["fl_date"] = pd.to_datetime(df["fl_date"], errors="coerce")

print("\nfl_date parse failed:")
print(df["fl_date"].isna().sum())


fl_date parse failed:
0


In [42]:
rebuild = pd.to_datetime(
    {
        "year": df["year"],
        "month": df["month"],
        "day": df["day_of_month"]
    },
    errors="coerce"
)

mask = df["fl_date"].notna() & rebuild.notna()

mismatch = df.loc[mask & (df["fl_date"] != rebuild)]

print("Date mismatch rows:", len(mismatch))


Date mismatch rows: 0


## Part 2

op_unique_carrier

op_carrier_fl_num

origin

origin_city_name

origin_state_nm

dest

dest_city_name

dest_state_nm

In [ ]:
# varible: op_unique_carrier

s = df["op_unique_carrier"].astype("string").str.strip()
mask_bad = s.isna() | (s.str.len() != 2)

In [ ]:
# every data point is good under the rule: no missing value no values are more than two characters
n_bad = int(mask_bad.sum())

In [ ]:
# varible: op_carrier_fl_num    

s_raw_op = df["op_carrier_fl_num"].astype("string").str.strip()
s = s_raw_op.mask(s_raw_op.eq(""), pd.NA)

# numeric for validation
num = pd.to_numeric(s, errors="coerce")


mask_missing = s.isna()
mask_not_int = (~mask_missing) & (num.isna() | (num % 1 != 0))
int_val = num.round(0).astype("Int64")

# no more than 4 digits
mask_gt_4_digits = (~mask_missing) & (~mask_not_int) & ((int_val < 0) | (int_val > 9999))

mask_bad = mask_missing | mask_not_int | mask_gt_4_digits

# summary counts

summary = pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(mask_missing.sum())],
        "not_int": [int(mask_not_int.sum())],
        "gt_4_digits": [int(mask_gt_4_digits.sum())],
        "bad_total": [int(mask_bad.sum())],
    }
)

# find one missing value


In [ ]:
# identify the missing value loction
missing_obs = df.loc[mask_missing].copy()
missing_obs["_op_carrier_fl_num_clean"] = s[mask_missing].to_numpy()

missing_obs #by checking flight history online, the missing value is 2483

In [ ]:
# input value 2483 in dataset
df["op_carrier_fl_num"] = int_val

df.loc[mask_missing, "op_carrier_fl_num"] = 2483


In [ ]:
# varible: origin  


# non-missing and exactly 3 characters 
s_raw_or = df["origin"].astype("string").str.strip()
s = s_raw_or.mask(s_raw_or.eq(""), pd.NA)

mask_missing = s.isna()
mask_not_3 = (~mask_missing) & (s.str.len() != 3)
mask_bad = mask_missing | mask_not_3

# summary

pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(mask_missing.sum())],
        "not_3_chars": [int(mask_not_3.sum())],
        "bad_total": [int(mask_bad.sum())],
    }
)

# all good



In [ ]:
# origin_city_name: "New York, NY" "Minneapolis, MN"
# to "New York", "Minneapolis"
# origin_state: "NY", "MN"

# check missing value: no missing

s_raw = df["origin_city_name"].astype("string").str.strip()
s = s_raw.mask(s_raw.eq(""), pd.NA)

mask_missing = s.isna()


pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(mask_missing.sum())],
    }
)


In [ ]:
# split stste name to a new varible: origin_state_abbr


s_raw1 = df["origin_city_name"].astype("string").str.strip()
origin_state_abbr = s_raw1.str.extract(r",\s*([A-Za-z]{2})\s*$", expand=False).str.upper()

df["origin_state_abbr"] = origin_state_abbr
df["origin_city_name"] = origin_state_abbr

# check
df[["origin_city_name", "origin_state_abbr"]].head()



In [ ]:
#check state name correction

us_state_abbr = [
    "AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA",
    "HI","ID","IL","IN","IA","KS","KY","LA","ME","MD",
    "MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ",
    "NM","NY","NC","ND","OH","OK","OR","PA","RI","SC",
    "SD","TN","TX","UT","VT","VA","WA","WV","WI","WY",
    "DC","PR", "VI", "TT"
]

abbr = df["origin_state_abbr"].astype("string").str.strip().str.upper()

mask_bad = abbr.isna() | (~abbr.isin(us_state_abbr))

pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(abbr.isna().sum())],
        "not_in_us_state_list": [int((~abbr.isna() & ~abbr.isin(us_state_abbr)).sum())],
        "bad_total": [int(mask_bad.sum())],
    }
)

# all good

In [ ]:
# origin_state_nm


s_raw_sta = df["origin_state_nm"].astype("string").str.strip()
s = s_raw_sta.mask(s_raw_sta.eq(""), pd.NA)

mask_missing = s.isna()


pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(mask_missing.sum())],
    }
)

# no missig value

In [ ]:
full_names = [
    "Texas","California","Florida","Illinois","Georgia","New York","Colorado","North Carolina","Virginia",
    "Arizona","Nevada","Washington","Michigan","Pennsylvania","Tennessee","Massachusetts","New Jersey",
    "Minnesota","Missouri","Hawaii","Utah","Maryland","Ohio","Oregon","Kentucky","Louisiana","South Carolina",
    "Indiana","Wisconsin","Oklahoma","Puerto Rico","Alaska","Alabama","Idaho","Arkansas","New Mexico",
    "Nebraska","Montana","Iowa","Connecticut","North Dakota","Maine","Rhode Island","Kansas","South Dakota",
    "Mississippi","Wyoming","Vermont","U.S. Virgin Islands","New Hampshire","West Virginia",
    "U.S. Pacific Trust Territories and Possessions",
]



mask_bad = s_raw_sta.isna() | (~s_raw_sta.isin(full_names))

pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(s_raw_sta.isna().sum())],
        "not_in_us_state_list": [int((~s_raw_sta.isna() & ~s_raw_sta.isin(full_names)).sum())],
        "bad_total": [int(mask_bad.sum())],
    }
)

# all good

In [ ]:
# varible: dest  


# non-missing and exactly 3 characters 
s_raw_d = df["dest"].astype("string").str.strip()
s = s_raw_d.mask(s_raw_d.eq(""), pd.NA)

mask_missing = s.isna()
mask_not_3 = (~mask_missing) & (s.str.len() != 3)
mask_bad = mask_missing | mask_not_3

# summary

pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(mask_missing.sum())],
        "not_3_chars": [int(mask_not_3.sum())],
        "bad_total": [int(mask_bad.sum())],
    }
)

# all good


In [ ]:
# dest_city_name: "Detroit, MI"
# to "Detroit"
# dest_state: "MI"

s_raw1 = df["dest_city_name"].astype("string").str.strip()
dest_state_abbr = s_raw1.str.extract(r",\s*([A-Za-z]{2})\s*$", expand=False).str.upper()


df["dest_state_abbr"] = dest_state_abbr
df["dest_city_name"] = dest_state_abbr

# check
df[["dest_city_name", "dest_state_abbr"]].head()



In [ ]:
#check state name

us_state_abbr = [
    "AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA",
    "HI","ID","IL","IN","IA","KS","KY","LA","ME","MD",
    "MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ",
    "NM","NY","NC","ND","OH","OK","OR","PA","RI","SC",
    "SD","TN","TX","UT","VT","VA","WA","WV","WI","WY",
    "DC","PR", "VI", "TT"
]

abbr = df["dest_state_abbr"].astype("string").str.strip().str.upper()

mask_bad = abbr.isna() | (~abbr.isin(us_state_abbr))

pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(abbr.isna().sum())],
        "not_in_us_state_list": [int((~abbr.isna() & ~abbr.isin(us_state_abbr)).sum())],
        "bad_total": [int(mask_bad.sum())],
    }
)

#all good

In [ ]:

# dest_state_nm


s_raw = df["dest_state_nm"].astype("string").str.strip()
s = s_raw.mask(s_raw.eq(""), pd.NA)

mask_missing = s.isna()


pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(mask_missing.sum())],
    }
)


full_names = [
    "Texas","California","Florida","Illinois","Georgia","New York","Colorado","North Carolina","Virginia",
    "Arizona","Nevada","Washington","Michigan","Pennsylvania","Tennessee","Massachusetts","New Jersey",
    "Minnesota","Missouri","Hawaii","Utah","Maryland","Ohio","Oregon","Kentucky","Louisiana","South Carolina",
    "Indiana","Wisconsin","Oklahoma","Puerto Rico","Alaska","Alabama","Idaho","Arkansas","New Mexico",
    "Nebraska","Montana","Iowa","Connecticut","North Dakota","Maine","Rhode Island","Kansas","South Dakota",
    "Mississippi","Wyoming","Vermont","U.S. Virgin Islands","New Hampshire","West Virginia",
    "U.S. Pacific Trust Territories and Possessions",
]



mask_bad = s_raw.isna() | (~s_raw.isin(full_names))

pd.DataFrame(
    {
        "total_rows": [len(df)],
        "missing": [int(s_raw.isna().sum())],
        "not_in_us_state_list": [int((~s_raw.isna() & ~s_raw.isin(full_names)).sum())],
        "bad_total": [int(mask_bad.sum())],
    }
)

# all good




## Part3

crs_dep_time

dep_time

dep_delay

taxi_out

wheels_off

wheels_on

taxi_in

crs_arr_time

arr_time

arr_delay

### part3_1

In [ ]:
cols3_1 = ["crs_dep_time","dep_time","dep_delay","taxi_out","taxi_in","cancelled"]

In [44]:
print("Missing crs_dep_time:")
print(df["crs_dep_time"].isna().sum())

Missing crs_dep_time:
0


In [46]:
mask_cancel = df["cancelled"] == 1

df.loc[mask_cancel, ["dep_delay","taxi_out","taxi_in"]] = np.nan

In [47]:
s = pd.to_numeric(df["crs_dep_time"], errors="coerce")
s_str = s.dropna().astype(int).astype(str).str.zfill(4)

df["crs_dep_time_tmp"] = pd.NA

df.loc[s_str.index, "crs_dep_time_tmp"] = (
    pd.to_datetime(s_str, format="%H%M", errors="coerce")
      .dt.strftime("%H:%M")
)


In [48]:
s = pd.to_numeric(df["dep_time"], errors="coerce")
s_str = s.dropna().astype(int).astype(str).str.zfill(4)

df["dep_time_tmp"] = pd.NA

df.loc[s_str.index, "dep_time_tmp"] = (
    pd.to_datetime(s_str, format="%H%M", errors="coerce")
      .dt.strftime("%H:%M")
)


In [49]:
df["crs_dep_time"] = df["crs_dep_time_tmp"]
df.drop(columns=["crs_dep_time_tmp"], inplace=True)

df["dep_time"] = df["dep_time_tmp"]
df.drop(columns=["dep_time_tmp"], inplace=True)


In [60]:
df["dep_time"] = pd.to_datetime(
    df["dep_time"],
    format="%H:%M",
    errors="coerce"
).dt.time
df["crs_dep_time"] = pd.to_datetime(
    df["crs_dep_time"],
    format="%H:%M",
    errors="coerce"
).dt.time

In [ ]:
crs = pd.to_numeric(df["crs_dep_time"], errors="coerce")
dep = pd.to_numeric(df["dep_time"], errors="coerce")

crs_h = crs // 100
crs_m = crs % 100
dep_h = dep // 100
dep_m = dep % 100

valid = (
    crs_h.between(0, 23) & crs_m.between(0, 59) &
    dep_h.between(0, 23) & dep_m.between(0, 59)
)

crs_min = crs_h * 60 + crs_m
dep_min = dep_h * 60 + dep_m

raw = dep_min - crs_min

mask = df["dep_delay"].notna() & valid

delta = df["dep_delay"] - raw

ok = (delta % 1440 == 0)

delay_mismatch = df.loc[mask & (~ok)]

print("dep_delay mismatch:", len(delay_mismatch))


dep_delay mismatch: 0


In [52]:

neg_taxi_out = df[df["taxi_out"] < 0]
neg_taxi_in  = df[df["taxi_in"] < 0]

print("\nNegative taxi_out:", len(neg_taxi_out))
print("Negative taxi_in:", len(neg_taxi_in))



Negative taxi_out: 0
Negative taxi_in: 0


### part3_2


In [92]:
s = pd.to_numeric(df["wheels_off"], errors="coerce")
s_str = s.dropna().astype(int).astype(str).str.zfill(4)

df["wheels_off_tmp"] = pd.NA

df.loc[s_str.index, "wheels_off_tmp"] = (
    pd.to_datetime(s_str, format="%H%M", errors="coerce")
      .dt.strftime("%H:%M")
)


In [93]:
s = pd.to_numeric(df["wheels_on"], errors="coerce")
s_str = s.dropna().astype(int).astype(str).str.zfill(4)

df["wheels_on_tmp"] = pd.NA

df.loc[s_str.index, "wheels_on_tmp"] = (
    pd.to_datetime(s_str, format="%H%M", errors="coerce")
      .dt.strftime("%H:%M")
)


In [94]:
s = pd.to_numeric(df["crs_arr_time"], errors="coerce")
s_str = s.dropna().astype(int).astype(str).str.zfill(4)

df["crs_arr_time_tmp"] = pd.NA

df.loc[s_str.index, "crs_arr_time_tmp"] = (
    pd.to_datetime(s_str, format="%H%M", errors="coerce")
      .dt.strftime("%H:%M")
)
     

In [95]:
s = pd.to_numeric(df["arr_time"], errors="coerce")
s_str = s.dropna().astype(int).astype(str).str.zfill(4)

df["arr_time_tmp"] = pd.NA

df.loc[s_str.index, "arr_time_tmp"] = (
    pd.to_datetime(s_str, format="%H%M", errors="coerce")
      .dt.strftime("%H:%M")
)
     

In [96]:
df["wheels_off"] = df["wheels_off_tmp"]
df.drop(columns=["wheels_off_tmp"], inplace=True)

df["wheels_on"] = df["wheels_on_tmp"]
df.drop(columns=["wheels_on_tmp"], inplace=True)

df["crs_arr_time"] = df["crs_arr_time_tmp"]
df.drop(columns=["crs_arr_time_tmp"], inplace=True)

df["arr_time"] = df["arr_time_tmp"]
df.drop(columns=["arr_time_tmp"], inplace=True)


In [97]:
df["wheels_off"] = pd.to_datetime(
    df["wheels_off"],
    format="%H:%M",
    errors="coerce"
).dt.time
df["wheels_on"] = pd.to_datetime(
    df["wheels_on"],
    format="%H:%M",
    errors="coerce"
).dt.time

df["arr_time"] = pd.to_datetime(
    df["arr_time"],
    format="%H:%M",
    errors="coerce"
).dt.time
df["crs_arr_time"] = pd.to_datetime(
    df["crs_arr_time"],
    format="%H:%M",
    errors="coerce"
).dt.time

In [98]:
crs = pd.to_numeric(df["crs_arr_time"], errors="coerce")
arr = pd.to_numeric(df["arr_time"], errors="coerce")

crs_h = crs // 100
crs_m = crs % 100
arr_h = arr // 100
arr_m = arr % 100

valid = (
    crs_h.between(0, 23) & crs_m.between(0, 59) &
    arr_h.between(0, 23) & arr_m.between(0, 59)
)

crs_min = crs_h * 60 + crs_m
arr_min = arr_h * 60 + arr_m

raw = arr_min - crs_min

mask = df["arr_delay"].notna() & valid

delta = df["arr_delay"] - raw

ok = (delta % 1440 == 0)

delay_mismatch = df.loc[mask & (~ok)]

print("arr_delay mismatch:", len(delay_mismatch))


arr_delay mismatch: 0


## Part 4

cancelled

cancellation_code

diverted 

### part6

In [53]:
cols6 = ["carrier_delay", "weather_delay", "nas_delay", "security_delay", "late_aircraft_delay"]


In [75]:
neg_mask = (df[cols6] < 0).any(axis=1)

neg_rows = df.loc[neg_mask, cols6]

print("Rows with negative delay components:", len(neg_rows))

Rows with negative delay components: 0


In [86]:
mask_arr = df["arr_delay"] > 0

sum_comp = df[cols6].sum(axis=1)

arr_delay_mismatch = mask_arr & (sum_comp != df["arr_delay"])

print("arr_delay sum mismatch rows:", arr_delay_mismatch.sum())


arr_delay sum mismatch rows: 1072038


In [ ]:
df["arr_delay_consistent"] = True
df.loc[mask_arr, "arr_delay_consistent"] = (
    sum_comp[mask_arr] == df.loc[mask_arr, "arr_delay"]
)

In [87]:

print("arr_delay > 0 rows:", mask_arr.sum())
print("consistent rows:",
      (mask_arr & (sum_comp == df["arr_delay"])).sum())
print("inconsistent rows:",
      (mask_arr & (sum_comp != df["arr_delay"])).sum())


arr_delay > 0 rows: 2522008
consistent rows: 1449970
inconsistent rows: 1072038


In [88]:
has_na_component = df[cols6].isna().any(axis=1)

(mask_arr & (sum_comp != df["arr_delay"]) & has_na_component).sum()


0

In [89]:
less = mask_arr & (sum_comp < df["arr_delay"])
greater = mask_arr & (sum_comp > df["arr_delay"])

print("sum < arr_delay:", less.sum())
print("sum > arr_delay:", greater.sum())


sum < arr_delay: 1072038
sum > arr_delay: 0


In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7079081 entries, 0 to 7079080
Data columns (total 36 columns):
 #   Column                Dtype         
---  ------                -----         
 0   year                  int64         
 1   month                 int64         
 2   day_of_month          int64         
 3   day_of_week           int64         
 4   fl_date               datetime64[ns]
 5   op_unique_carrier     object        
 6   op_carrier_fl_num     float64       
 7   origin                object        
 8   origin_city_name      object        
 9   origin_state_nm       object        
 10  dest                  object        
 11  dest_city_name        object        
 12  dest_state_nm         object        
 13  crs_dep_time          object        
 14  dep_time              object        
 15  dep_delay             float64       
 16  taxi_out              float64       
 17  wheels_off            float64       
 18  wheels_on             float64       
 19  